# Module 3. Introduction to Convolutional Neural Networks (CNNs)
In the previous chapter, we discussed the challenges faced by the FCN when applied to the CIFAR-10 dataset, despite its relatively good performance on the MNIST dataset. Due to the complexity of the CIFAR-10 dataset and its images with multiple classes and slightly higher resolution, more advanced techniques are required to produce accurate predictions.

This chapter will explore how Convolutional Neural Networks (CNNs) can help us overcome these difficulties. CNNs are particularly effective in image classification tasks and work by processing data in a hierarchical way, allowing them to capture more complex features. By utilizing CNNs, we can expect to achieve higher accuracy and more reliable results.

Overall, this chapter provides an in-depth examination of how CNNs can be implemented to address the challenges posed by the CIFAR-10 dataset, and how they can lead to more accurate and reliable results

## Convolutional Layer
A convolutional layer is the fundamental component of a convolutional neural network. Its primary role is to identify the features of an image that are critical to the objects it represents. The layer consists of a feature-finding matrix that scans the image, pixel by pixel, to extract meaningful features. These features are the building blocks that identify the objects in the image. By analyzing these features, the network can identify and classify the objects with greater accuracy and efficiency. The process of identifying the features can be further enhanced by adjusting the parameters of the matrix, such as the filter size (see example later) and [stride](https://medium.com/analytics-vidhya/convolution-padding-stride-and-pooling-in-cnn-13dc1f3ada26). This allows the layer to learn more complex features that are not apparent in the raw image data. Ultimately, the convolutional layer is critical to the success of the convolutional neural network, as it provides the foundation for accurate object recognition and classification.

### Convolutions
Convolution is a math way to mix two functions and make a new one. In Convolutional Neural Networks (CNNs), the two functions are the input image and the convolutional filter. By using convolutions, a new image can be made with different pixel values and the convolutional filter.

In the example below, we demonstrate how to apply a filter to an image to generate a new image that mostly contains vertical lines and removes the horizontal ones (although some horizontal lines may still appear in the new image).

![An example of applying a convolution filter](./convolution.jpg)

When you add convolutional layers to a network, you increase the number of parameters that need to be learned. This, in turn, increases the time and space complexity of the learning process. To address this issue, pooling layers are used. Pooling reduces the size of the network by reducing the number of parameters passed to the next layer. The pooling operation applies a summary statistical function, such as maximum or average, to resize its input and reduce the overall number of parameters passed to the next layer while keeping the features.

The pooling layer downsamples the feature maps produced by the convolutional layer into a smaller number of parameters, thus reducing computational complexity. It is common practice to add pooling layers after every one or two convolutional layers in the CNN architecture.

## Implementing a CNN for the signed dataset
If you remember what happened in the previous notebook (module_fcnn), the fully connected neural network in use wasn't quite successful in classifying our data. Let's see the potential of CNNs for this case. For doing that, the next two cells deal with downloading the data and loading it into data generators. If you have downloaded the data already, run only the second one. If you aren't sure you did, run both.


In [ ]:
!wget https://drive.google.com/uc?id=1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR --output-document train.cvs
!wget https://drive.google.com/uc?id=1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg --output-document test.cvs

In [ ]:
import csv
import string
import numpy as np
import tensorflow as tf


def parse_data_from_input(filename):
  with open(filename) as file:
    csv_reader = csv.reader(file, delimiter=',')
    labels = []
    images = []

    #ignore first row
    next(csv_reader)
    for line in csv_reader:
      labels.append(line[0])
      images.append(line[1:])
    
    labels = np.array(labels)
    labels = labels.astype(np.float64)
    images = np.reshape(images,(-1,28,28))
    images = images.astype(np.float64)

    return images, labels


from tensorflow.keras.preprocessing.image import ImageDataGenerator
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
  training_images   = np.expand_dims(training_images,3)
  validation_images = np.expand_dims(validation_images,3)

  training_labels = tf.keras.utils.to_categorical(training_labels,26)
  train_datagen   = ImageDataGenerator(rescale=1.0/255.0)

  train_generator = train_datagen.flow(x=training_images,
                                       y=training_labels,
                                       batch_size=32) 
  
  validation_labels  = tf.keras.utils.to_categorical(validation_labels,26)
  validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

  validation_generator = validation_datagen.flow(x=validation_images,
                                                 y=validation_labels,
                                                 batch_size=32) 
  return train_generator, validation_generator



training_images, training_labels = parse_data_from_input("train.cvs")
validation_images, validation_labels = parse_data_from_input("test.cvs")
train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)



Now we are going to define our convolutional neural network. Actually, you will do it yourself. In particular, we would like to have the following architecture:
- A convolutional layer with 16 filters of size 3 x 3 and relu activation
- A max pooling layer
- A convolutional layer with 32 filters of size 3 x 3 and reul ativation
- A max pooling layer
- A dense layer of 128 units and relu activation
- A dense layer with 26 units (as many as categories) and softmax ativation

  Check on the Keras documentation for layers Conv2D, and MaxPooling2D in order to understand the parameters that you need. 


In [ ]:
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(26,activation='softmax'),                                   
                                     
  ])
  

  model.compile(optimizer = tf.optimizers.RMSprop(learning_rate=0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])




In [ ]:
model.summary()
history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

That looks much nicer, doesn't it? However, there is still a gap between what the model can achieve on the training data and what the model can achieve on the validation one. 

## Not enough data? Data augmenttion 

A reason for the behaiour above might be that the images on the validation set contain features that are arranged differently than the images in the training set. To account for this possibility, we are going to work with a technique called data augmentation. 

The idea would be simulating that we have a larger dataset. This increase inn size gives the model more images to learn from while training. The increase in variance helps the model ignore unimportant features and select only the features that are truly important in classification, allowing it to generalize better.

Before we use the image data generator of Keras, in order to create a mechanism to feed images to our model and perform some basic operations (such as normalisation). In addition to these things, the generators allow performing some basic transformation into the images. We recommend checking out the [documentation here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator). It accepts a series of options for augmenting your data. 

Modify the generator in order to take advantae of this and see whether you can impove the results computed by the model you created before (do not forget to reset the model as otherwise you are simply further training it.)



In [ ]:
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
  training_images   = np.expand_dims(training_images,3)
  validation_images = np.expand_dims(validation_images,3)

  training_labels = tf.keras.utils.to_categorical(training_labels,26)
  train_datagen   = ImageDataGenerator(rescale=1.0/255.0,
                                      rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                                       zoom_range=0.1,  # Randomly zoom image
                                       width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                       height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                       horizontal_flip=True,  # randomly flip images horizontally
                                       vertical_flip=False)
    

  train_generator = train_datagen.flow(x=training_images,
                                       y=training_labels,
                                       batch_size=32) 
  
  validation_labels  = tf.keras.utils.to_categorical(validation_labels,26)
  validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

  validation_generator = validation_datagen.flow(x=validation_images,
                                                 y=validation_labels,
                                                 batch_size=32) 
  return train_generator, validation_generator

train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)


In [ ]:
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(26,activation='softmax'),                                   
                                     
  ])
  

  model.compile(optimizer = tf.optimizers.RMSprop(learning_rate=0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])


history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()